# New topo DEMs in Southwest WA

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from clawpack.geoclaw import topotools
from clawpack.visclaw import geoplot
from importlib import reload
import geopandas as gpd
import folium
import pandas as pd

In [ ]:
import find_topo_source

In [ ]:
eighth = 1/8  # half tile width
yrange = arange(47.0, 47.25+eighth, 0.25)
xrange = arange(-124.25, -123.5+eighth, 0.25)
name = 'NCEI tiles around Grays Harbor'
find_topo_source.make_combined_tile_kml(name, xrange, yrange)

In [ ]:
reload(find_topo_source)

In [ ]:
tile_names = []
for x in xrange:
    for y in yrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        print(f'midpoints: {xm:.3f}, {ym:.3f}, NW corner:  {x:.3f}, {y:.3f}, tile: {tile_name}')
        tile_names.append(tile_name)

### map of tiles

In [ ]:
gdf = gpd.read_file('NCEI_tiles_around_Grays_Harbor.kml', driver='KML')
m = folium.Map(location=(47,-124), zoom_start=9, scrollWheelZoom=False)
folium.GeoJson(gdf).add_to(m)

for x in xrange:
    for y in yrange:
        xm = x+eighth
        ym = y-eighth
        tile_name = find_topo_source.tile_coords(xm,ym,verbose=False)
        folium.Marker(
            location=[y-eighth,x+eighth],
            #popup = f"{x:.3f}, {y:.3f}: \n<b>Tile:</b>\n {tile_name}",
            popup = f"<b>Tile:</b>\n {tile_name}",
            tooltip="Click for info",
            icon=folium.Icon(color="red")  #, icon="cloud") # Customize the marker's appearance
        ).add_to(m) 
m

## Versions of these tiles found on NCEI webpages:

In [ ]:
for tile_name in tile_names:
    print('\n-----------------------------------')
    tile_urls = find_topo_source.find_tile_url(tile_name, verbose=True)
    if len(tile_urls) == 0:
        print(f'{tile_name} not found')

## Compare versions of n47x00_w124x25

Three versions were found on the NCEI websites, labelled `2018v1.nc`, `2025v1.nc` and `2025v2.tif`.
We also have a modified version from the Westport Maritime study that corrected the topo around the marina to include the seawalls better.

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/sowa_mhw_19_n47x00_w124x25_2025v1.nc'
tile0_2025v1 = topotools.read_netcdf(url)

In [ ]:
tile0_2025v1.crop(coarsen=18).plot(limits=(-100,50))

In [ ]:
def read_tif(url):
    import rasterio
    with rasterio.open(url) as src:
        print(f"Coordinate Reference System (CRS): {src.crs}")
        print(f"Bounds: {src.bounds}")
        print(f"Number of bands: {src.count}")
        bounds = src.bounds
        print(f"Reading Z data array with shape {src.width}x{src.height}...")
        topotif = src.read()
    Z = flipud(topotif[0,:,:])
    x1,x2,y1,y2 = extent = [bounds.left, bounds.right, bounds.bottom, bounds.top]
    dx = (x2-x1)/Z.shape[0]
    dy = (y2-y1)/Z.shape[1]
    print(f'dx = {dx:.7f} degrees = {dx*3600:.7f} arcseconds')
    print(f'dy = {dy:.7f} degrees = {dy*3600:.7f} arcseconds')
    
    x = arange(x1+0.5*dx, x2, dx)
    y = arange(y1+0.5*dy, y2, dy)
    
    topo = topotools.Topography()
    topo.set_xyZ(x,y,Z)
    return topo

In [ ]:
url = 'https://coast.noaa.gov/htdata/raster2/elevation/NCEI_ninth_Topobathy_2014_8483/wash_outercoast/ncei19_n47x00_w124x25_2025v2.tif'
tile0_2025v2 = read_tif(url)

In [ ]:
dx = dy = 1/(9*3600)
print(f'tile0_2025v1.x alignment: {mod((tile0_2025v1.x[0]+124.)/dx, 1):.3f} * dx')
print(f'tile0_2025v1.y alignment: {mod((tile0_2025v1.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'tile0_2025v2.x alignment: {mod((tile0_2025v2.x[0]+124.)/dx, 1):.3f} * dx')
print(f'tile0_2025v2.y alignment: {mod((tile0_2025v2.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
tile0_2025v1_fcn = tile0_2025v1.make_function()
tile0_2025v2_fcn = tile0_2025v2.make_function()

In [ ]:
y0 = 47.03
xtrans = linspace(-124.11, -124.104, 1000)
ytrans = linspace(46.905, 46.909, 1000)
dz_mhw = 2.3  # approximate correction NAVD88 to MHW
fig,ax = subplots(figsize=(10,5))
plot(xtrans, tile0_2025v1_fcn(xtrans,ytrans), 'g', label='tile0_2025v1')
plot(xtrans, tile0_2025v2_fcn(xtrans,ytrans) - dz_mhw, 'r', label='tile0_2025v2')
grid(True)
#ylim(-3,10.)
#xlim(-124.18, -124.1)
#ylim(-3,6)
legend(framealpha=1)
title(f'Transect through Westport Marina');

## Compare versions of n47x50_w124x50

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/tiled_19as/ncei19_n47x50_w0124x50_2018v1.nc'
topo1 = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/ncei19_n47x50_w0124x50_2020.nc'
topo1a = topotools.read_netcdf(url)

In [ ]:
print(topo1.x[:3], topo1.x[-3:])
print(topo1.y[:3], topo1.y[-3:])

In [ ]:
print(topo1a.x[:3], topo1a.x[-3:])
print(topo1a.y[:3], topo1a.y[-3:])

In [ ]:
dx = dy = 1/(9*3600)
print(f'topo1.x alignment: {mod((topo1.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo1.y alignment: {mod((topo1.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo1a.x alignment: {mod((topo1a.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo1a.y alignment: {mod((topo1a.y[0]-47.)/dy, 1):.3f} * dx')

## Compare versions of n47x25_w124x25

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/tiled_19as/ncei19_n47x25_w0124x25_2018v1.nc'
topo2 = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/ncei19_n47x25_w0124x25_2020.nc'
topo2a = topotools.read_netcdf(url)

In [ ]:
url = 'https://www.ngdc.noaa.gov/thredds/dodsC/tiles/nthmp/tiled_19as/sowa_mhw_19_n47x25_w124x25_2025v1.nc'
topo2b = topotools.read_netcdf(url)

In [ ]:
print(f'topo2.x alignment: {mod((topo2.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2.y alignment: {mod((topo2.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo2a.x alignment: {mod((topo2a.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2a.y alignment: {mod((topo2a.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
print(f'topo2b.x alignment: {mod((topo2b.x[0]+124.)/dx, 1):.3f} * dx')
print(f'topo2b.y alignment: {mod((topo2b.y[0]-47.)/dy, 1):.3f} * dx')

In [ ]:
topo2f = topo2.make_function()
topo2af = topo2a.make_function()
topo2bf = topo2b.make_function()

In [ ]:
y0 = 47.03
xtrans = arange(-124.2, -124, dx/5)
ytrans = y0 * ones(xtrans.shape)
dz_mhw = 2.3  # approximate correction NAVD88 to MHW
fig,ax = subplots(figsize=(10,5))
plot(xtrans, topo2f(xtrans,ytrans) - dz_mhw, 'g', label='topo2 - 2018 adjusted to MHW')
plot(xtrans, topo2af(xtrans,ytrans), 'r', label='topo2a - 2020')
plot(xtrans, topo2bf(xtrans,ytrans), 'b', label='topo2b - 2025')
grid(True)
#ylim(-3,10.)
xlim(-124.18, -124.1)
ylim(-3,6)
legend(framealpha=1)
title(f'Transect through Ocean Shores at y = {y0:.5f}');

In [ ]:
dz = topo2f(xtrans,ytrans) - topo2af(xtrans,ytrans)
dz.min(), dz.max()